In [1]:
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import meshio, numpy, copy
import meshplot as mp
import sys
import pathlib
# sys.path.append(str(pathlib.Path("/home/zizhou/smooth-ipc/build") / "debug" / "python"))  # noqa

from ipctk import *

In [2]:
V = np.array([[0.5,0],[1.0,0],[1.5,0],[1.01,0.05],[1.02, 0.06]])
E = np.array([[1,0],[2,1],[4,3]], dtype=int)

F = np.array([])

dhat=0.11
param = ParameterType(dhat, 1, 0, 0.2, 0, 1, 200)
mesh = CollisionMesh(V, E, F)

In [3]:
# single point
def potential_single_point(points):
    cs2 = SmoothCollisions2()
    cs2.use_high_order_quadrature = False
    cs2.build(mesh, points, param)
    B = SmoothPotential(param)
    return B(cs2, mesh, points)

# IPC
def potential_IPC(points):
    cs3 = Collisions()
    cs3.build(mesh, points, param.dhat)
    B = BarrierPotential(param.dhat)
    return B(cs3, mesh, points)

In [7]:
displacements = np.linspace(-0.2, 0.2, 1000)
pA = []
pB = []
pC = []

for i, disp in enumerate(displacements):
    V_disp = copy.deepcopy(V)
    V_disp[[3, 4], 0] += disp
    
    pA.append(potential_IPC(V_disp))
    pB.append(potential_single_point(V_disp))


fig = go.Figure(data=[
    go.Scatter(x=displacements, y=pA, name="IPC"),
    go.Scatter(x=displacements, y=pB, name="Single Point"),
], layout=go.Layout(width=800, height=400))

fig.update_layout(
    yaxis_type="log"
)

fig.show()

In [ ]:
xs = numpy.linspace(0.4, 1.6, 400)
ys = numpy.linspace(0.001, 0.1, 200)
dtypes = numpy.empty((ys.size, xs.size), dtype=float)
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        V_disp = copy.deepcopy(V)
        V_disp[3, :] = [x, y]
        V_disp[4, :] = [x + 0.01, y + 0.01]
        dtypes[j, i] = potential_single_point(V_disp)

In [6]:
fig = go.Figure(data=[
    go.Scatter(x=V[[0,1,2], 0], y=V[[0,1,2], 1],
        marker=dict(
            color='red',       # Dot color
            size=10
        ),
        line=dict(
            color='red',      # Line color
            width=2
        )),
    go.Scatter(x=V[[3,4], 0], y=V[[3,4], 1],
        marker=dict(
            color='red',       # Dot color
            size=10
        ),
        line=dict(
            color='red',      # Line color
            width=2
        )),
    go.Contour(z=dtypes, x=xs, y=ys,
        contours=dict(
            start=0,   # minimum contour level
            end=50,      # maximum contour level
            size=5    # step between levels
        ))
])
fig.show()